In [1]:
import pandas as pd
import numpy as np

In [2]:
matches = pd.read_csv('/Users/tliu/Desktop/Erdos Project/Data/matches.csv')

In [3]:
matches.head()

,player1,player2,score1,score2,best_of,tournament_id,date,year
0,Colin Roscoe,Jackie Rea,9,6,17,753,NaN,1982
1,Tommy Murphy,Clive Everton,9,4,17,753,NaN,1982
2,Vic Harris,Marcus Owen,9,4,17,753,NaN,1982
3,Bob Harris,Graham Cripsey,9,6,17,753,NaN,1982
4,Geoff Foulds,Matt Gibson,9,3,17,753,NaN,1982


In [4]:
#Using matches history of before 202x to predict matches in 202x
from elo_v3 import elo_rating
from sklearn.metrics import accuracy_score

tournaments = list(matches.tournament_id.unique())

nth = [750, 800, 1080]
scores = np.zeros((3,1))


i=0
for n in nth:
    for j in range(1):
        past_tournaments = tournaments[:n]
        current = tournaments[n]
        past_index = matches['tournament_id'].map(lambda x: x in past_tournaments)
        

        matches_past = matches[past_index]
        matches_future = matches[matches['tournament_id'] == current]

        #Update statistics of players
        elo = elo_rating()
        elo.update_stats(matches = matches_past)

        #Store the actual result in y
        matches_future['result'] = 0  
        y = np.array(matches_future['result'])

        #Making prediction
        X = matches_future[['player1', 'player2', 'best_of']]
        prediction, _ = elo.predict(X)

        #Record the accuracy score
        from sklearn.metrics import accuracy_score
        scores[i,j] = accuracy_score(prediction, y)
        
        j += 1
    i+=1




/Users/tliu/Desktop/Erdos Project/Elo Rating/elo_v3/elo_v3.py:120: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.stats_by_years.fillna(0, inplace = True)
/var/folders/pd/b57yb11s57b55h2t3r3c8q1r0000gn/T/ipykernel_3168/1055904842.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_future['result'] = 0
/Users/tliu/Desktop/Erdos Project/Elo Rating/elo_v3/elo_v3.py:120: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(co

In [5]:
print(scores)

[[0.72222222]
 [0.71428571]
 [0.73239437]]


The prediction accuracies for 750th, 800th and 1080th are 0.72, 0,71 and 0.73.
Recall the average accuracies using rankings are 0.56031746, 0.5952381 , 0.49577465. See details in 'Baseline/baseline1/baseline1.ipynb'.  
Our elo system has significantly higher prediction accuracy.